In [ ]:
#! If you want to force use of CPU (before import of tensorflow)
import os
# os.environ['CUDA_VISIBLE_DEVICES']='-1'
# os.environ['OMP_NUM_THREADS'] = '4'
# os.environ['export OPENBLAS_NUM_THREADS']='4'

#! import as tf
import tensorflow as tf

#! If you want to limit number of threads, the mechine has 32 virtual cores (16 physical ones)
print('thread inter = ', tf.config.threading.get_inter_op_parallelism_threads() )
print('thread intra = ', tf.config.threading.get_intra_op_parallelism_threads() )
 
tf.config.threading.set_inter_op_parallelism_threads(4)
tf.config.threading.set_intra_op_parallelism_threads(4)

#! If you want to limit the GPU memory usage, the machine has ~24GB. For most applications a few GBs is enough.
#! If you max out what you specify progressively up the threshhold.
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.set_logical_device_configuration(
    physical_devices[0],
    [tf.config.LogicalDeviceConfiguration(memory_limit=10_000)])

  logical_devices = tf.config.list_logical_devices('GPU')

except:
  # Invalid device or cannot modify logical devices once initialized.
  pass

#! Will print out what devices you are using
tf.config.list_physical_devices()



import tensorflow_probability as tfp
import numpy as np
import matplotlib.pyplot as plt
import time
from tqdm import tqdm

from mlkernels.tensorflow import Matern32, EQ, Matern12, Matern52


from RGNP.NeuralNet import NeuralN
from RGNP.utilities import pshape
from RGNP.utilities import reparameterize, log_normal_pdf, expHighLowFunc_01
from RGNP.GICNet    import FFLayer, KIPLayer, GICNet

pi  = tf.constant(np.pi, tf.float32)
tfd = tfp.distributions
tfb = tfp.bijectors


dirname='data-pddlvm-burgers/'
os.makedirs(dirname, exist_ok=True)



In [ ]:
dimU = 1
dimZ = 2
dimW = 1
dimX = 2
                         #  x         t
domain     = tf.constant( [[0., 1.], [0., 1.]] )

nxSamples  = 225
resolution = tf.constant([15,15])

#currently in dataset
zDist = tfd.Uniform(low=tf.constant([0.01, 0.5], shape=[dimZ]), high=tf.constant([0.1, 1.], shape=[dimZ]))
wDist = tfd.Uniform(low=tf.constant([0.5], shape=[dimW]), high=tf.constant([2.], shape=[dimW]))

plt.hist(zDist.sample(sample_shape=(50))[:,0], bins=20)
plt.hist(zDist.sample(sample_shape=(50))[:,1], bins=20)
plt.hist(wDist.sample(sample_shape=(50))[:,0], bins=20)
plt.show()
plt.close()

e_bound = 1e-5

def raveled2DDomainGrid(resolution):
    x, t          = tf.linspace(domain[0][0], domain[0][1], resolution[0]), tf.linspace(domain[1][0], domain[1][1], resolution[1])
    X, T          = np.meshgrid(x, t, indexing='ij')
    Xr, Tr        = tf.reshape(tf.constant(X, dtype=tf.float32), [-1,1]), tf.reshape(tf.constant(T, dtype=tf.float32), [-1,1])
    XTr           = tf.concat([Xr, Tr], 1)
    return XTr


b      = zDist.sample()
gridXTr = raveled2DDomainGrid(resolution)
print('gridXTr.shape', gridXTr.shape)
    

In [ ]:
# Define alpha network

dim_DeepO_out  = dimU * 2

alphaNet  = NeuralN('alpha', np.array([ 1, 1, dimZ]) )

numFreq = 100

nhAlpha = 400

activ_alpha = 'swish'

alphaNet.NN = tf.keras.Sequential([ 
                                tf.keras.layers.Dense( nhAlpha, input_dim=( numFreq * 2 + dimZ + dimW ), activation=activ_alpha),
                                tf.keras.layers.Dense( nhAlpha, activation=activ_alpha),
                                tf.keras.layers.Dense( nhAlpha, activation=activ_alpha),
                                tf.keras.layers.Dense( nhAlpha, activation=activ_alpha),
                                tf.keras.layers.Dense( nhAlpha, activation=activ_alpha),
                                tf.keras.layers.Dense( nhAlpha, activation=activ_alpha),
                                tf.keras.layers.Dense( nhAlpha, activation=activ_alpha),
                                tf.keras.layers.Dense( nhAlpha, activation=activ_alpha),
                                tf.keras.layers.Dense( nhAlpha, activation=activ_alpha),
                                tf.keras.layers.Dense( dim_DeepO_out ) ])

freqsX = tf.linspace(0., 1., numFreq)

freqs_x  = expHighLowFunc_01(2., 0.01, freqsX)[None,:]
freqs_t  = expHighLowFunc_01(2., 0.01, freqsX)[None,:]
freqs    = tf.concat([freqs_x, freqs_t], 1)

#now they are trainable
alphaNet.fLayer = [ FFLayer( freqs, trainable=True ) ]

alphaNet.lower_alpha = tf.constant(1e-4)
alphaNet.upper_w     = tf.constant(1.)

def alphaMap(self, z, w, x):

        z_tile  = tf.tile(tf.reshape(z, [1,-1]), [x.shape[0], 1])
        w_tile  = tf.tile(tf.reshape(w, [1,-1]), [x.shape[0], 1])
        print('x.shape', x.shape)
        x       = tf.reshape(x, [-1,dimX])

        xFreq   = self.fLayer[0]( x )
        x_input = tf.concat([xFreq, z_tile, w_tile], 1)

        print('x_input.shape', x_input.shape)
        mapped  = self.NN( x_input )
        
        mean, logvar = mapped[:,:dimU], mapped[:,dimU:2*dimU]
        
        tf.debugging.assert_all_finite(mean, 'mean not finite')
        tf.debugging.assert_all_finite(logvar, 'logvar not finite')

        logvar = self.dBoundwalpha_log( logvar )

        print('alpha mean.shape = ',   mean.shape)
        print('alpha logvar.shape = ', logvar.shape)

        return mean, logvar

from types import MethodType
alphaNet.call = MethodType(alphaMap, alphaNet)
alphaNet(tf.constant(1., shape=[dimZ]), tf.constant(1., shape=[dimW]), gridXTr)
alphaNet.compile()
alphaNet.NN.summary()
alphaNet.summary()


# alphaNet.save_weights(dirname + '/alphaModel/alphaModel-lfmDiag')
# alphaNet.load_weights(dirname + '/alphaModel/alphaModel-lfmDiag')

# del alphaNet


In [ ]:
# Define beta network



dv_beta       = 20 # number of channels -- filters
dim_out_beta  = 100 #dimZ #20 #works



convLayers = tf.keras.Sequential( [ 
                        tf.keras.layers.Conv2D( dv_beta, (2), strides=(1), activation='swish' ),
                        tf.keras.layers.Conv2D( dv_beta, (2), strides=(1), activation='swish' ),
                        tf.keras.layers.Conv2D( dv_beta, (2), strides=(1), activation='swish' ),
                        tf.keras.layers.Conv2D( dv_beta, (2), strides=(1), activation='swish' ),
                        tf.keras.layers.Conv2D( dv_beta, (2), strides=(1), activation='swish' ),
                        tf.keras.layers.Conv2D( dv_beta, (2), strides=(1), activation='swish' ),
                        tf.keras.layers.Flatten(  ),
                        tf.keras.layers.Dense(500),
                        tf.keras.layers.Dense(500),
                        tf.keras.layers.Dense(500),
                        tf.keras.layers.Dense(dim_out_beta)
                        ] )
                        
betaNet    = NeuralN('beta', np.array([ 1, 1, dimZ]) )
betaNet.NN = convLayers

betaNet.lower_alpha  = tf.constant(1e-4) 
betaNet.upper_w      = tf.constant(1.)

def betaMap(self, u, w, x):

        x   = tf.reshape( x, [resolution[0], resolution[1], dimX ] )
        u   = tf.reshape( u, [resolution[0], resolution[1], dimU ] )
        Ws  = tf.tile( tf.reshape(w, [1,1,dimW]), [resolution[0], resolution[1], 1] )
        input = tf.concat([x, Ws, u], -1)

        mapped = self.NN( input[None,...] )
        print('mapped.shape = ', mapped.shape)

        mean, logvar = mapped[:,:dimZ], mapped[:,dimZ:2*dimZ]

        tf.debugging.assert_all_finite(mean, 'mean not finite')
        tf.debugging.assert_all_finite(logvar, 'logvar not finite')

        logvar = self.dBoundwalpha_log( logvar )
        print('beta mean.shape = ', mean.shape)
        print('beta logvar.shape = ', logvar.shape)

        return mean, logvar

from types import MethodType
betaNet.call = MethodType(betaMap, betaNet)

Ws  = tf.tile( tf.reshape(tf.constant(1., shape=[dimW]), [1,1,dimW]), [resolution[0], resolution[1], 1] )

mean_alpha, logvar_alpha = alphaNet(tf.constant(1., shape=[dimZ]), tf.constant(1., shape=[dimW]), gridXTr)
print(mean_alpha.shape)

mean_beta, logvar_beta = betaNet( mean_alpha, tf.constant(1., shape=[dimW]), gridXTr )
betaNet.compile( )
betaNet.summary( )
betaNet.NN.summary( )



# betaNet.save_weights(dirname + '/betaModel/betaModel-lfmDiag')
# betaNet.load_weights(dirname + '/betaModel/betaModel-lfmDiag')

# del betaNet


In [ ]:

def D_x(x):
    return tf.sin(pi * x)

def initU(x, w):
    return tf.sin(2. * tf.reshape(w, []) * pi * x) * D_x(x)

def D_t(t):
    return t

alphaNet.epsilon_r = tf.Variable(1., trainable=True)

def residualFunction( z, w, xt ):
    
    x, t = xt[:, :1], xt[:, 1:]
    print('x,shape', x.shape)
    print('t,shape', t.shape)

    z = tf.reshape(z, [-1])
    w = tf.reshape(w, [ ])

    with tf.GradientTape(watch_accessed_variables=False, persistent=True) as t1:
        t1.watch( x )
        t1.watch( t )
        with tf.GradientTape(watch_accessed_variables=False, persistent=True) as t2:
            t2.watch( x )
            t2.watch( t )

            mean_alpha, logvar_alpha = alphaNet( z, w, tf.concat([x,t], 1) )
            u_untrans = tf.reshape(reparameterize(mean_alpha, logvar_alpha), [-1,1])

            u = initU(x, w) + D_x(x) * D_t(t) * u_untrans
            
        u_x = tf.reshape(t2.gradient(u, x), [-1,1])
        u_t = tf.reshape(t2.gradient(u, t), [-1,1])

    u_xx = tf.reshape(t1.gradient(u_x, x), [-1,1])
    
    res = u_t +  z[1] * u * u_x  - z[0] * u_xx 


    tf.debugging.assert_all_finite(res, 'res not finite')

    
    print("residualBody.shape", res.shape)
    print('u.shape', u.shape)
    print('mean_alpha.shape', mean_alpha.shape)
    print('logvar_alpha.shape', logvar_alpha.shape )
        
    logP_alpha_u_Z = log_normal_pdf(u_untrans, mean_alpha, logvar_alpha, raxis=0)

    return u_untrans, u, res, logP_alpha_u_Z


def ELBO( zw ):

    z,w    = zw[:dimZ], zw[dimZ:dimZ+dimW]

    print('ELBO z.shape', z.shape)
    print('ELBO w.shape', w.shape)

    u_untrans, us, residual, logP_alpha_u_Z = residualFunction( z, w, gridXTr )

    logVarEpsVect  = tf.repeat(2.*tf.math.log(alphaNet.epsilon_r), residual.shape[0] )[:, None]

    logP_r   = log_normal_pdf( residual, tf.constant(0., shape=residual.shape), logVarEpsVect, raxis=0 )
    
    tf.debugging.assert_all_finite(logP_r, 'logP_r not finite')

    mean_beta, logvar_beta = betaNet( tf.reshape(u_untrans, [-1,1]), w, gridXTr )

    print('mean_beta.shape', mean_beta.shape)
    print('mean_beta.shape', logvar_beta.shape)

    logP_beta_Z_u          = log_normal_pdf( tf.reshape(z, [1,-1]), mean_beta, logvar_beta, raxis=1 ) #this is correct dimension

    print('logP_alpha_u_Z.shape', logP_alpha_u_Z.shape)
    print('logP_beta_Z_u.shape', logP_beta_Z_u.shape)
    print('logP_r.shape', logP_r.shape)

    elbo = tf.reduce_mean( logP_r + logP_beta_Z_u -  logP_alpha_u_Z )

    return elbo

zs =  zDist.sample( )
ws =  wDist.sample( )

ELBO( tf.concat([zs, ws], 0) )

In [ ]:


trainableVars   = [ alphaNet.trainable_variables, betaNet.trainable_variables]

@tf.function
def train_step(optimizer):

    zs =  zDist.sample( batchSize )
    ws =  wDist.sample( batchSize )

    print('zs.shape', zs.shape)
    print('ws.shape', ws.shape)

    zws = tf.concat([zs, ws], 1)

    print('zwxs.shape', zws.shape)

    with tf.GradientTape(persistent=True) as tape:

        losses = tf.vectorized_map(ELBO, zws, fallback_to_while_loop=False)
        loss   = - tf.reduce_mean(losses)
        tf.debugging.assert_all_finite(loss, 'loss not finite')
        
    for vars in trainableVars:
        gradients = tape.gradient(loss, vars)
        optimizer.apply_gradients(zip(gradients, vars))

    return loss

batchSize = 25

num_iterations   = tf.constant(2_000_000)
intervalELBOsave = 1000



lr = tf.keras.optimizers.schedules.ExponentialDecay(
    1e-3, int(num_iterations/batchSize/5), 0.5, staircase=True, name=None
)
optimizer = tf.keras.optimizers.Adam( lr )


elboAll     = []
epsAll      = []
startTrainingPDDLVM = time.time()
for iteration in tf.range(0, int(num_iterations/batchSize) ):

    start_time = time.time()
    loss = train_step(optimizer)

    end_time = time.time()

    if iteration % intervalELBOsave == 0:
        epsAll.append( alphaNet.epsilon_r.numpy() )
        elboAll.append( -loss ) 
        print('Epoch: {}, Test set ELBO: {:.2f}, epsidlon_r {:.4f}, time elapse for current epoch: {:.4f}'
        .format(iteration, -loss, alphaNet.epsilon_r.numpy(), end_time - start_time))

endTrainingPDDLVM = time.time()
totalTimeTrainingPDLVM = (endTrainingPDDLVM - startTrainingPDDLVM)/60.

print('totalTimeTrainingPDLVM = {:.2f}m'.format( totalTimeTrainingPDLVM ) )



In [ ]:
ebloAll = tf.stack(elboAll)
LOSS    = np.array(ebloAll)
plt.plot(range(LOSS.shape[0]), LOSS )
plt.show()
print(np.isnan(LOSS).mean())
plt.plot(range(LOSS.shape[0]), np.log(-LOSS) )

stack_epsAll = tf.stack(epsAll)
plt.plot(range( len(epsAll)), stack_epsAll )
plt.show()

np.savetxt(dirname  + 'ebloAll.dat' , ebloAll )
np.savetxt(dirname  + 'epsAll.dat'  , stack_epsAll )
print('totalTimeTrainingPDLVM = {:.2f}m'.format( totalTimeTrainingPDLVM ) )


In [ ]:
z = [0.01, 1.]
w = [2.]

# z = [1., 1.]
# w = [1.]

resolution = [200,200]

X, T = np.meshgrid( tf.linspace(0., 1., 200), tf.linspace(0., 1., 200), indexing='ij')

xr, tr = tf.reshape(X, [-1,1]), tf.reshape(T, [-1,1])
mean_alpha, logvar_alpha = alphaNet( z, w, tf.concat([xr, tr], 1) )
u_untrans = tf.reshape(reparameterize(mean_alpha, logvar_alpha), [-1,1])

u = initU(xr, w) + D_x(xr) * D_t(tr) * u_untrans

plt.contourf(X, T, tf.reverse(tf.reshape(u, resolution), [0]), 50)
plt.colorbar()
plt.show()


In [ ]:
dirnameGT = '../../../fenics/Burgers/data-groundTruth-burgers_1000'

groundTruthU  = tf.constant( np.load(dirnameGT+'/' + 'SolutionsUGT.npy' ), dtype=tf.float32)[:, 1:-1:1,1::1]
allTrueZs     = tf.constant( np.loadtxt(dirnameGT+'/' + 'allZ.dat' ), dtype=tf.float32)
allTrueWs     = tf.constant( np.loadtxt(dirnameGT+'/' + 'allW.dat' ), dtype=tf.float32)
groundTruthXT = tf.constant( np.load(dirnameGT+'/' + 'FenicsMeshXT.npy'     ), dtype=tf.float32)[:, 1:-1:1,1::1]

print('groundTruthU.shape', groundTruthU.shape)
print('allTrueZs.shape', allTrueZs.shape)
print('allTrueWs.shape', allTrueWs.shape)
print('groundTruthXT.shape', groundTruthXT.shape)

nx = groundTruthXT.shape[1]
nt = groundTruthXT.shape[2]




saveDataList = True

if saveDataList:

     allresAbsMean = []

     allSampleZs   = []
     allMeanZs     = []
     allStdZs      = []

     allUSamples = []
     allUMeans   = []
     allUStd     = []

     allZNSE       = []
     allUNSE       = []

     Pin2SigK = []
     Pin2SigU = []

print('groundTruthXT.shape', groundTruthXT.shape)

Xtest, Ttest = groundTruthXT[0], groundTruthXT[1]

xtest, ttest = tf.reshape(Xtest, [-1,1]), tf.reshape(Ttest, [-1,1])
xttest_rav   = tf.concat([xtest, ttest], 1)

tf_residualFunction = tf.function( residualFunction )#residualFunction #
tf_alphaNet         = tf.function( alphaNet         )#alphaNet         #
tf_betaNet          = tf.function( betaNet          )#betaNet          #

PLOT = False
savePlots   = False
BenchParams = False
writeToFile = True

resolution = tf.constant([15,15])
kip = KIPLayer(EQ(), 1./tf.cast(resolution[0], tf.float32))

for i in range(groundTruthU.shape[0]):
     
     zTrue = tf.reshape( allTrueZs[i], [-1, 1] )
     wTrue = tf.reshape( allTrueWs[i], [ ] )
     uTrue = tf.reshape( tf.reverse(groundTruthU[i], [0]), [-1, 1] )

     if BenchParams:
          zTrue = tf.constant([0.01, 1.])
          wTrue =  tf.constant([2.])
     
     
     mean_alpha, logvar_alpha = tf_alphaNet( zTrue, wTrue, xttest_rav )
     u_untrans = tf.reshape(reparameterize(mean_alpha, logvar_alpha), [-1,1])

     u = initU(xtest, wTrue) + D_x(xtest) * D_t(ttest) * u_untrans
     mean_alpha_trans   = tf.squeeze( initU(xtest, wTrue) + D_x(xtest) * D_t(ttest) * mean_alpha )
     var_alpha_trans    = tf.squeeze( (D_x(xtest) * D_t(ttest))**2. * tf.exp(logvar_alpha) )
     std_alpha_trans    = tf.sqrt( var_alpha_trans )

     if PLOT:
          plt.rcParams.update({
          "text.usetex": True,
          "font.family": "sans-serif",
          "font.sans-serif": ["Helvetica"],
          'axes.labelsize': 16,
          'axes.titlesize': 16,
          'xtick.labelsize' : 16,
          'ytick.labelsize' : 16
                    })
          # latex font definition
          plt.rc('text', usetex=True)
          plt.rc('font', **{'family':'serif','serif':['Computer Modern Roman']})
          
          plt.contourf(Xtest, Ttest, tf.reshape(u, [nx, nt]), 50)
          plt.ylabel(r'$t$')
          plt.xlabel(r'$x$')
          plt.colorbar()
          plt.tight_layout()
          plt.savefig(dirname+'/'+'uSample_{}.png'.format(i))
          if savePlots: np.save(dirname+'/'+'uSample_{}.png'.format(i), u)
          plt.show() 

          plt.contourf(Xtest, Ttest, tf.reshape(mean_alpha_trans, [nx, nt]), 50)
          plt.ylabel(r'$t$')
          plt.xlabel(r'$x$')
          plt.colorbar()
          plt.tight_layout()
          plt.savefig(dirname+'/'+'mean_alpha_trans_{}.png'.format(i))
          if savePlots: np.save(dirname+'/'+'mean_alpha_trans_{}.png'.format(i), mean_alpha_trans)
          plt.show() 

          plt.contourf(Xtest, Ttest, tf.reshape(std_alpha_trans, [nx, nt]), 50)
          plt.ylabel(r'$t$')
          plt.xlabel(r'$x$')
          plt.colorbar()
          plt.tight_layout()
          plt.savefig(dirname+'/'+'std_alpha_trans{}.png'.format(i))
          if savePlots: np.save(dirname+'/'+'std_alpha_trans{}.png'.format(i), std_alpha_trans)
          plt.show() 

          plt.contourf(Xtest, Ttest, tf.reshape(uTrue, [nx, nt]), 50)
          plt.ylabel(r'$t$')
          plt.xlabel(r'$x$')
          plt.tight_layout()
          plt.colorbar()
          plt.savefig(dirname+'/'+'uTrue{}.png'.format(i))
          if savePlots: np.save(dirname+'/'+'uTrue{}.png'.format(i), uTrue)
          plt.show()  

          plt.contourf(Xtest, Ttest, tf.reshape(uTrue, [nx, nt]) - tf.reshape(mean_alpha_trans, [nx,nt]), 50)
          plt.ylabel(r'$t$')
          plt.xlabel(r'$x$')
          plt.tight_layout()
          plt.colorbar()
          plt.savefig(dirname+'/'+'uTrue-u{}.png'.format(i))
          if savePlots: np.save(dirname+'/'+'uTrue-u{}.png'.format(i), tf.reshape(uTrue, [nx, nt]) - tf.reshape(mean_alpha_trans, [nx,nt]))
          plt.show()  

     u_true_unstranf        = (uTrue - initU(xtest, wTrue)) / ( D_x(xtest) * D_t(ttest) )

     interpedU = kip.call(xttest_rav, tf.reshape(u_true_unstranf, [-1,1]), gridXTr)

     mean_beta, logvar_beta = tf_betaNet( tf.reshape(interpedU, [-1,1]), wTrue,  gridXTr )

     mean_beta, logvar_beta = tf.squeeze( mean_beta), tf.squeeze( logvar_beta )
     sampleZ  = reparameterize( mean_beta, logvar_beta )
     std_beta = tf.exp( 0.5 * logvar_beta)

     print('mean_beta = ', mean_beta)
     print('std_beta = ' , tf.exp( 0.5 * logvar_beta))
     print('true z = '   , zTrue)
     print('true w = '   , wTrue)
     print('\n\n')


     if saveDataList:

          allSampleZs.append(tf.squeeze(sampleZ))
          allMeanZs.append(tf.squeeze(mean_beta))
          allStdZs.append(std_beta)

          allUSamples.append(tf.squeeze(u))
          allUMeans.append(tf.squeeze(mean_alpha_trans))
          allUStd.append(tf.squeeze(std_alpha_trans))
          
          allZNSE.append( ( tf.linalg.norm((tf.squeeze(mean_beta) - tf.squeeze(zTrue)) ) / tf.linalg.norm(tf.squeeze(zTrue)) )**2. )

          allUNSE.append( ( tf.linalg.norm((tf.squeeze(mean_alpha_trans) - tf.squeeze(uTrue)) ) / tf.linalg.norm(tf.squeeze(uTrue)) )**2. )

          Pin2SigK.append( tf.reduce_sum(tf.where(tf.math.logical_and( 
               tf.squeeze(zTrue) > mean_beta-2*std_beta, tf.squeeze(zTrue) < mean_beta+2*std_beta
               ), 1, 0)) / zTrue.shape[0] )

          Pin2SigU.append( tf.reduce_sum(tf.where(tf.math.logical_and(
               tf.squeeze(uTrue) > mean_alpha_trans-2*std_alpha_trans, tf.squeeze(uTrue) < mean_alpha_trans+2*std_alpha_trans
               ), 1, 0)) / uTrue.shape[0]  )


print('mean allZNSE = ', tf.reduce_mean( tf.stack(allZNSE) ) )
print('mean allUNSE = ', tf.reduce_mean( tf.stack(allUNSE) ) )

print('sttdev allZNSE = ', tf.math.reduce_std( tf.stack(allZNSE) ) )
print('sttdev allUNSE = ', tf.math.reduce_std( tf.stack(allUNSE) ) )

print('Pin2SigK = ', tf.reduce_mean(tf.stack(Pin2SigK)))
print('Pin2SigU = ', tf.reduce_mean(tf.stack(Pin2SigU)))

print('allUStd = ', tf.reduce_mean(tf.stack(allUStd)))

if writeToFile:
     with  open(dirname+"allInfoRun.txt", "w") as f:
          f.write('mean allZNSE = {}\n'.format(tf.reduce_mean( tf.stack(allZNSE) )) )
          f.write('mean allUNSE = {}\n'.format(tf.reduce_mean( tf.stack(allUNSE) )) )
          f.write('sttdev allZNSE = {}\n'.format(tf.math.reduce_std( tf.stack(allZNSE) )) )
          f.write('sttdev allUNSE = {}\n'.format(tf.math.reduce_std( tf.stack(allUNSE) )) )
          f.write('Pin2SigK = {}\n'.format(tf.reduce_mean(tf.stack(Pin2SigK))) )
          f.write('Pin2SigU = {}\n'.format(tf.reduce_mean(tf.stack(Pin2SigU))) )
          f.write('totalTimeTrainingPDLVM = {:.2f}m\n'.format( totalTimeTrainingPDLVM ))
     f.close()

print('totalTimeTrainingPDLVM = {:.2f}m'.format( totalTimeTrainingPDLVM ) )



raise

In [ ]:
if True:
    alphaNet.save_weights(dirname + '/alphaModel/alphaModel-lfmDiag')
    # alphaNet.load_weights(dirname + '/alphaModel/alphaModel-lfmDiag')

    # del alphaNet

if True:
    betaNet.save_weights(dirname + '/betaModel/betaModel-lfmDiag')
    # betaNet.load_weights(dirname + '/betaModel/betaModel-lfmDiag')

    # del betaNet


In [ ]:
saveData = True
if saveData:

    allSampleZs_np = tf.stack(allSampleZs)
    allMeanZs_np   = tf.stack(allMeanZs)
    allStdZs_np    = tf.stack(allStdZs)

    allUSamples_np  = tf.stack(allUSamples)
    allUMeans_np    = tf.stack(allUMeans)
    allUStd_np      = tf.stack(allUStd)

    prefix = 'randgrid{}_{}kIter'.format(nxSamples, int(num_iterations/1000))

    np.savetxt(dirname + prefix + '_allRes_np.dat'   , tf.stack(tf.squeeze(allresAbsMean)) )


    np.savetxt(dirname + prefix + '_allSampleZs_np.dat', allSampleZs_np)
    np.savetxt(dirname + prefix + '_allMeanZs_np.dat'  , allMeanZs_np)
    np.savetxt(dirname + prefix + '_allStdZs_np.dat'  , allMeanZs_np)


    np.savetxt(dirname + prefix + '_allUSamples_np.dat' , allUSamples_np)
    np.savetxt(dirname + prefix + '_allUMeans_np.dat'   , allUMeans_np)
    np.savetxt(dirname + prefix + '_allUStd_np.dat'     , allUStd_np)
    
    np.savetxt(dirname + prefix + '_allZNSE_np.dat'   , tf.stack( allZNSE ) )
    np.savetxt(dirname + prefix + '_allUNSE_np.dat'   , tf.stack( allUNSE ) )

In [ ]:
for layer in betaNet.NN.kipLayers:
    print(layer.ell)